<h2>Coleta de dados de Wiki</h2>

Este é um projeto de estudo, o objetivo é aplicar conhecimentos de <em>webscrap</em> para coletar dados dispostos em wikis sobre o jogo Arknights, desenvolvido pela Hypergryph.

O jogo possui uma gama de personagens com diferentes níveis de poder e impacto para o sucesso no mesmo. As wikis consultadas possuem diversas discussões e comentários acerca da utilidade de cada personagem que podem auxiliar na priorização de utilização e evolução.

Origem dos dados: https://arknights.fandom.com/


In [26]:
# Módulos instalados no ambiente

# pip install requests
# pip install beautifulsoup4
# pip install tqdm
# pip install fastparquet
# pip install pyarrow

In [2]:
import requests
import pandas as pd
from tqdm import tqdm
from bs4 import BeautifulSoup

In [4]:
headers = {
    "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
    "accept-language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7,es;q=0.6,zh-CN;q=0.5,zh;q=0.4",
    "cache-control": "max-age=0",
    "priority": "u=0, i",
    "referer": "https://arknights.fandom.com/wiki/Operator/List",
    "sec-ch-ua": '"Not/A)Brand";v="8", "Chromium";v="126", "Google Chrome";v="126"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "document",
    "sec-fetch-mode": "navigate",
    "sec-fetch-site": "same-origin",
    "sec-fetch-user": "?1",
    "upgrade-insecure-requests": "1",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36",
}

base_url = "https://arknights.fandom.com/"
response = requests.get(base_url + "wiki/Operator/6-star", headers=headers)
# Resgatando informações de personagens de uma certa raridade para análise inicial

In [5]:
print(response.status_code)
# response.text # Comentando para prevenir poluir o notebook com todo o conteúdo da página

200


A análise do HTML da página foi realizada tanto utilizando o resultado fornecido pelo BeautifulSoup quanto com a utilização do inspecionar no navegador.


In [6]:
if response.status_code == 200:
    page_html = BeautifulSoup(response.text)
# page_html # Comentado para evitar poluir o notebook

Inicialmente foi procurado a estrutura a partir do nome dos primeiros personagens, e onde eles possivelmente apareceriam listados. Foi encontrado que os mesmos estavam dispostas em uma estrutura <code>table</code> o que é facilmente identificável ao visualizar a página.


In [7]:
def get_character_list_data(html_text):
    table_operators = html_text.find("table", class_="sortable")
    table_operators = table_operators.find("tbody")
    table_rows = table_operators.find_all("tr")

    columns_th = table_rows[0].find_all("th")
    columns_names = [column.text for column in columns_th]
    columns_names = columns_names[1:]  # A primeira coluna é a imagem do personagem

    table_data = []

    for row in table_rows[1:]:
        reference_list = row.find_all("a")

        new_element = {}
        # Referenciando as informações de acordo com a ordem das colunas na tabela
        for index, reference in enumerate(reference_list):
            if index == 0:
                new_element["reference"] = reference.attrs["href"]

            new_element[columns_names[index]] = reference.attrs["title"]

        table_data.append(new_element)

    return table_data


result = get_character_list_data(page_html)
result[0]

{'reference': '/wiki/Aak',
 'Name': 'Aak',
 'Class': 'Specialist',
 'Branch': 'Geek',
 'Faction': "Lee's Detective Agency"}

Após essa função temos as informações mais básicas dos personagens de determinada categoria, neste caso personagens marcados como "6 estrelas".


In [8]:
# Poderia realizar a busca de categorias de forma automática também resgatando de outra página
# Repassando os valores diretos para simplificar pois dificilmente serão adicionadas novas raridades
operator_list_prefix = "wiki/Operator/"
rarity_list = ["6-star", "5-star", "4-star", "3-star", "2-star", "1-star"]


def get_from_rarity_list(prefix, rarity_list):
    all_rarities_list = []
    for rarity in tqdm(rarity_list):
        # Requisita para a página especifica daquela raridade
        full_url = base_url + prefix + rarity
        response = requests.get(full_url, headers=headers)
        if response.status_code == 200:
            rarity_response = get_character_list_data(BeautifulSoup(response.text))
            for resp in rarity_response:
                resp["rarity"] = rarity
        else:
            print(f"Requisição não foi completada {response.status_code} para {rarity}")

        all_rarities_list.append(rarity_response)

    return all_rarities_list


data = get_from_rarity_list(operator_list_prefix, rarity_list)
print(len(data))
print(len(data[0]))
print(data[0][:3])

100%|██████████| 6/6 [00:01<00:00,  3.17it/s]

6
81
[{'reference': '/wiki/Aak', 'Name': 'Aak', 'Class': 'Specialist', 'Branch': 'Geek', 'Faction': "Lee's Detective Agency", 'rarity': '6-star'}, {'reference': '/wiki/Angelina', 'Name': 'Angelina', 'Class': 'Supporter', 'Branch': 'Decel Binder', 'Faction': 'Siracusa', 'rarity': '6-star'}, {'reference': '/wiki/Archetto', 'Name': 'Archetto', 'Class': 'Sniper', 'Branch': 'Marksman', 'Faction': 'Laterano', 'rarity': '6-star'}]


Agora temos a uma lista de listas, contendo todos os personagens de diferentes raridades. Em seguida foi utilizado apenas um compressor de listas para tornar a lista em uma única dimensão com o registro de todos os personagens.


In [10]:
flat_data = [element for elements in data for element in elements]

flat_data
print(len(flat_data))
print(flat_data[0])
print(flat_data[-1])

296
{'reference': '/wiki/Aak', 'Name': 'Aak', 'Class': 'Specialist', 'Branch': 'Geek', 'Faction': "Lee's Detective Agency", 'rarity': '6-star'}
{'reference': '/wiki/U-Official', 'Name': 'U-Official', 'Class': 'Supporter', 'Branch': 'Bard', 'Faction': 'Rhodes Island', 'rarity': '1-star'}


Com a listagem em um único level temos a estrutura de um DataFrame que pode ser aplicado e analisado pela biblioteca Pandas.


In [38]:
characters = pd.DataFrame(flat_data)
characters.head()

,reference,Name,Class,Branch,Faction,rarity,Global?\n
0,/wiki/Aak,Aak,Specialist,Geek,Lee's Detective Agency,6-star,NaN
1,/wiki/Angelina,Angelina,Supporter,Decel Binder,Siracusa,6-star,NaN
2,/wiki/Archetto,Archetto,Sniper,Marksman,Laterano,6-star,NaN
3,/wiki/Ash,Ash,Sniper,Marksman,Team Rainbow,6-star,NaN
4,/wiki/Bagpipe,Bagpipe,Vanguard,Charger,Victoria,6-star,NaN


In [39]:
characters.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 296 entries, 0 to 295
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   reference  296 non-null    object
 1   Name       296 non-null    object
 2   Class      296 non-null    object
 3   Branch     296 non-null    object
 4   Faction    296 non-null    object
 5   rarity     296 non-null    object
 6   Global?
   1 non-null      object
dtypes: object(7)
memory usage: 16.3+ KB


In [45]:
characters.groupby(by="Class").size()

Class
Caster        40
Defender      34
Guard         58
Kestrel        1
Medic         27
Sniper        47
Specialist    33
Supporter     28
Vanguard      28
dtype: int64

In [27]:
characters.to_parquet("operatores_data", index=False)